In [2]:
import requests
import zipfile
import os

# URL for the MovieLens Tag Genome Dataset 2021
url = 'https://files.grouplens.org/datasets/movielens/ml-25m.zip'
output_dir = 'database'

# Download the dataset
response = requests.get(url)
zip_path = 'ml-25m.zip'
with open(zip_path, 'wb') as f:
    f.write(response.content)

# Extract the dataset
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(output_dir)

# Cleanup the zip file
os.remove(zip_path)

print(f'Dataset extracted to {output_dir}')


KeyboardInterrupt: 

In [2]:
import sqlite3

def create_database():
    # Kết nối đến cơ sở dữ liệu SQLite
    path = "db.sqlite3"
    conn = sqlite3.connect(path)
    print("Database connection established:", conn)

    # Tạo con trỏ để thực thi các lệnh SQL
    cursor = conn.cursor()

    # Lệnh SQL để tạo các bảng
    sql_commands = [
        """
        CREATE TABLE IF NOT EXISTS Users (
            user_id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            email TEXT NOT NULL UNIQUE,
            password TEXT NOT NULL,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            role TEXT NOT NULL
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS Categories (
            category_id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            description TEXT,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS Movies (
            movieId INTEGER PRIMARY KEY AUTOINCREMENT,
            title TEXT NOT NULL,
            director TEXT,
            actors TEXT,
            summary TEXT,
            rating REAL,
            price REAL,
            release_date DATE,
            poster_url TEXT,
            category_id INTEGER,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (category_id) REFERENCES Categories(category_id)
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS Orders (
            order_id INTEGER PRIMARY KEY AUTOINCREMENT,
            user_id INTEGER,
            total_price REAL,
            order_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            status TEXT NOT NULL,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (user_id) REFERENCES Users(user_id)
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS OrderItems (
            order_item_id INTEGER PRIMARY KEY AUTOINCREMENT,
            order_id INTEGER,
            movieId INTEGER,
            quantity INTEGER NOT NULL,
            price REAL NOT NULL,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (order_id) REFERENCES Orders(order_id),
            FOREIGN KEY (movieId) REFERENCES Movies(movieId)
        );
        """
    ]

    try:
        # Thực thi từng lệnh SQL để tạo các bảng
        for command in sql_commands:
            cursor.execute(command)
            print(f"Executed SQL command: {command}")
        print("Tables created successfully.")
    except sqlite3.Error as e:
        print(f"An error occurred: {e}")

    # Đóng kết nối
    conn.commit()  # Lưu các thay đổi
    conn.close()
    print("Database connection closed.")

if __name__ == "__main__":
    create_database()


Database connection established: <sqlite3.Connection object at 0x0000022CF0C08220>
Executed SQL command: 
        CREATE TABLE IF NOT EXISTS Users (
            user_id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            email TEXT NOT NULL UNIQUE,
            password TEXT NOT NULL,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            role TEXT NOT NULL
        );
        
Executed SQL command: 
        CREATE TABLE IF NOT EXISTS Categories (
            category_id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            description TEXT,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        );
        
Executed SQL command: 
        CREATE TABLE IF NOT EXISTS Movies (
            movie_id INTEGER PRIMARY KEY AUTOINCREMENT,
            title TEXT NOT NULL,
            director TEXT,
   

In [5]:
import sqlite3
import pandas as pd

def create_tables(conn):
    sql_commands = [
        """
        CREATE TABLE IF NOT EXISTS Users (
            user_id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            email TEXT NOT NULL UNIQUE,
            password TEXT NOT NULL,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            role TEXT NOT NULL
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS Categories (
            category_id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            description TEXT,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS Movies (
            movieId INTEGER PRIMARY KEY AUTOINCREMENT,
            title TEXT NOT NULL,
            director TEXT,
            actors TEXT,
            summary TEXT,
            rating REAL,
            price REAL,
            release_date DATE,
            poster_url TEXT,
            category_id INTEGER,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (category_id) REFERENCES Categories(category_id)
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS Orders (
            order_id INTEGER PRIMARY KEY AUTOINCREMENT,
            user_id INTEGER,
            total_price REAL,
            order_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            status TEXT NOT NULL,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (user_id) REFERENCES Users(user_id)
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS OrderItems (
            order_item_id INTEGER PRIMARY KEY AUTOINCREMENT,
            order_id INTEGER,
            movieId INTEGER,
            quantity INTEGER NOT NULL,
            price REAL NOT NULL,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (order_id) REFERENCES Orders(order_id),
            FOREIGN KEY (movieId) REFERENCES Movies(movieId)
        );
        """
    ]

    cursor = conn.cursor()
    for command in sql_commands:
        cursor.execute(command)
    conn.commit()

def load_csv_to_sqlite(table_name, csv_path, conn):
    df = pd.read_csv(csv_path)
    df.to_sql(table_name, conn, if_exists='append', index=False)

def main():
    # Đường dẫn đến cơ sở dữ liệu SQLite
    db_path = "movie_db.sqlite3"
    conn = sqlite3.connect(db_path)
    
    # Tạo các bảng
    create_tables(conn)

    # Nạp dữ liệu từ các tệp CSV vào các bảng tương ứng
    load_csv_to_sqlite('Movies', 'database/movies.csv', conn)

    # Đóng kết nối
    conn.close()

if __name__ == "__main__":
    main()


OperationalError: table Movies has no column named movieId